In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.spatial.distance as dist
import matplotlib as mpl
import iblofunmatch.inter as ibfm
output_dir="output"

In [ ]:
from navground import sim, core

yaml = """
steps: 10000
time_step: 0.1
save_directory: ''
record_pose: true
record_twist: true
scenario:
  type: Cross
  agent_margin: 0.2
  side: 28
  target_margin: 0.6
  tolerance: 0.4
  groups:  
    -
      type: thymio
      number: 20
      control_period: 0.1
      behavior:
        type: HL
        safety_margin: 0.2 
        horizon: 3
        barrier_angle: 1
      radius: 0.2
      kinematics:
        type: 2WDiff
        max_speed: 1.0
        wheel_axis: 2
      state_estimation:
        type: Bounded
        range: 2.0 
"""
experiment = sim.load_experiment(yaml)
experiment.run()

In [ ]:
run = experiment.runs[0]
ps = run.poses[:,:,[0,1]]
twists = run.twists[:,:,:2] # ignore angular speeds

In [ ]:
weight = 4

In [ ]:
def trajectory_distance_weighted_velocities(positions, velocities, weight):
    assert(len(positions)>0)
    assert(len(positions)==len(velocities))
    positions_velocities_list = []
    for idx, points in enumerate(positions):
        positions_velocities_list.append(np.hstack((points, velocities[idx]*weight)))
    distances_list = []
    for idx, points_vel in enumerate(positions_velocities_list):
        # Compare trajectories at same time
        points_vel_compare = positions_velocities_list[idx]
        distances_list.append(dist.cdist(points_vel, points_vel_compare, "minkowski", p=2))
    # end for
    distances_arr = np.array(distances_list)
    return np.min(distances_arr, axis=0)

In [ ]:
dist_result = trajectory_distance_weighted_velocities(ps[:5], twists[:5], weight)

In [ ]:
def distances_weighted_velocities(points, velocities, weight):
    points_velocities = np.hstack((points, velocities*weight))
    return dist.squareform(dist.pdist(points_velocities, "minkowski", p=2))

In [ ]:
def plot_matching_diagram_trajectories(positions, velocities, weight, ax, shift_time, color="blue"):
    # Compute X, Z barcodes and matching
    half_shift=int(shift_time/2)
    Dist_X = trajectory_distance_weighted_velocities(positions[:half_shift], velocities[:half_shift], weight)
    Dist_Y = trajectory_distance_weighted_velocities(positions, velocities, weight)
    Dist_Z = np.minimum(Dist_X, Dist_Y)
    idx_S = list(range(Dist_X.shape[0]))
    # Compute matching from X to Z
    ibfm_out = ibfm.get_IBloFunMatch_subset(Dist_X, Dist_Z, idx_S, output_dir, max_rad=-1, num_it=1, store_0_pm=True, points=False, max_dim=1)
    # Plot 0 persistence diagram of matching 
    match_diagram = []
    for idx, bar_X in enumerate(ibfm_out["S_barcode_0"]):
        idx_match = ibfm_out["induced_matching_0"][idx]
        bar_Z = ibfm_out["X_barcode_0"][idx_match]
        match_diagram.append([bar_X[1], bar_Z[1]])
    # end for
    match_diagram = np.array(match_diagram)
    # Plot matching diagram
    ax.scatter(match_diagram[:,0], match_diagram[:,1], color=color)
    ax.plot([0,np.max(match_diagram)*1.1], [0,np.max(match_diagram)*1.1], color="gray")

In [ ]:
start_step_list = list(range(7300,7800, 100))
fig, ax = plt.subplots(ncols=len(start_step_list), figsize=(6*(len(start_step_list)),6))
xmax, ymax = 0,0
for idx_start, start_step in enumerate(start_step_list):
    shift_time=100
    steplist = list(range(start_step, start_step+shift_time*2, 5))
    for idx, start_step in enumerate(steplist):
        plot_matching_diagram_trajectories(ps[start_step:start_step+shift_time], twists[start_step:start_step+shift_time], weight, ax[idx_start], shift_time, color=mpl.colormaps["GnBu"](idx/len(steplist)))
    ax[idx_start].set_title(f"{start_step}")
    xmax = max(xmax, ax[idx_start].get_xlim()[1])
    ymax = max(ymax, ax[idx_start].get_ylim()[1])

for idx_start, start_step in enumerate(start_step_list):
    ax[idx_start].plot([0,max(xmax, ymax)], [0, max(xmax, ymax)], color="gray")
    ax[idx_start].set_xlim(0,xmax)
    ax[idx_start].set_ylim(0,ymax)

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
start_step=5495
shift_time =100
plot_matching_diagram_trajectories(ps[start_step:start_step+shift_time], twists[start_step:start_step+shift_time], weight, ax, shift_time, color="blue")
# get data
positions = ps[start_step:start_step+shift_time]
velocities = twists[start_step:start_step+shift_time]
half_shift = int(shift_time/2)
Dist_X = trajectory_distance_weighted_velocities(positions[:half_shift], velocities[:half_shift], weight)
Dist_Y = trajectory_distance_weighted_velocities(positions, velocities, weight)
Dist_Z = np.minimum(Dist_X, Dist_Y)
idx_S = list(range(Dist_X.shape[0]))
# Compute matching from X to Z
ibfm_out = ibfm.get_IBloFunMatch_subset(Dist_X, Dist_Z, idx_S, output_dir, max_rad=-1, num_it=1, store_0_pm=True, points=False, max_dim=1)
# put persistence pairs together
match_diagram = []
for idx, bar_X in enumerate(ibfm_out["S_barcode_0"]):
    idx_match = ibfm_out["induced_matching_0"][idx]
    bar_Z = ibfm_out["X_barcode_0"][idx_match]
    match_diagram.append([bar_X[1], bar_Z[1]])
# end for
match_diagram = np.array(match_diagram)

print(np.array(match_diagram))
print(ibfm_out["S_reps_0"])

In [ ]:
def plot_sequence(X_list, ax, mark_points=[], color="red"):
    # Plot figure
    X_old = X_list[0]
    ax.scatter(X_old[:,0], X_old[:,1], s=20, marker="o", color=mpl.colormaps["RdBu"](1/(len(X_list)+1)), zorder=2)
    for idx, X in enumerate(X_list[1:]):
        ax.scatter(X[:,0], X[:,1], s=20, marker="o", color=mpl.colormaps["RdBu"]((idx+1)/(len(X_list)+1)), zorder=2, label="X")
        # for edge in zip(X, X_old):
        #     edge_pts = np.array(edge)
        #     ax.plot(edge_pts[:,0], edge_pts[:,1], color="gray", zorder=1)
        if len(mark_points)>0:
            mark_X = X[mark_points]
            ax.scatter(mark_X[:,0], mark_X[:,1], s=20, marker="+", color=color, zorder=3)
        X_old = X
    #end for 

In [ ]:
X_seq = ps[list(range(start_step, start_step+shift_time+1, 2))]
len(X_seq)
fig, ax = plt.subplots(figsize=(10, 5))
ax.set_aspect("equal")
plot_sequence(X_seq, ax, mark_points=[3], color="yellow")